<a href="https://colab.research.google.com/github/Abhinavpanjiyar1/Pytorch/blob/main/pytorch_understanding_rnn_shape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# N = no. of samples
T = sequence of length
D =  number of input features
M = number of hiddent units
K = number of output units


In [2]:
# make some data
N = 1
T =10
D = 3
M = 5
K = 2
X = np.random.randn(N, T, D)



In [6]:
#  make an rnn
class SimpleRNN(nn.Module):
  def __init__(self, n_inputs, n_hidden, n_outputs):
    super().__init__()
    self.D =n_inputs
    self.M =n_hidden
    self.K =n_outputs
    self.rnn = nn.RNN(input_size =self.D,
                      hidden_size =self.M,
                      nonlinearity ='tanh',
                      batch_first =True)
    self.fc =nn.Linear(self.M, self.K)

  def forward(self, X):
    # initiate hidden state
    h0 = torch.zeros(1, X.size(0), self.M)

    # get rnn unit output
    out,_ = self.rnn(X,h0)

    # we only want h(T) at final time step
    out =self.fc(out)
    return out




In [7]:
#instantiate the model
model =SimpleRNN(n_inputs =D, n_hidden = M, n_outputs = K)

In [8]:
inputs = torch.from_numpy(X.astype(np.float32))
out = model(inputs)
out

tensor([[[ 0.4804, -0.1704],
         [ 1.0260, -0.5907],
         [ 0.5872,  0.1412],
         [ 0.9235, -0.3137],
         [ 0.8062, -0.3233],
         [ 1.0268, -0.4661],
         [ 0.8037, -0.3812],
         [ 0.5748, -0.0902],
         [ 0.5966, -0.2572],
         [ 0.8854, -0.2608]]], grad_fn=<ViewBackward0>)

In [9]:
out.shape

torch.Size([1, 10, 2])

In [10]:
Yhats_torch =out.detach().numpy()

In [11]:
W_xh, W_hh, b_xh, b_hh =model.rnn.parameters()

In [12]:
W_xh.shape

torch.Size([5, 3])

In [13]:
W_xh

Parameter containing:
tensor([[ 0.1126, -0.1347, -0.3630],
        [-0.3188, -0.2909,  0.2552],
        [ 0.1780,  0.3703,  0.1422],
        [-0.3138,  0.2415,  0.2166],
        [ 0.3425,  0.1999,  0.2666]], requires_grad=True)

In [14]:
W_xh = W_xh.data.numpy()
W_xh

array([[ 0.11258831, -0.13471636, -0.36301553],
       [-0.31879443, -0.29085723,  0.25516286],
       [ 0.17795551,  0.3703343 ,  0.14219472],
       [-0.31384736,  0.24150087,  0.21657857],
       [ 0.34251556,  0.19994484,  0.2665565 ]], dtype=float32)

In [15]:
W_hh =W_hh.data.numpy()
b_xh = b_xh.data.numpy()
b_hh = b_hh.data.numpy()

In [16]:
W_xh.shape, W_hh.shape, b_xh.shape, b_hh.shape

((5, 3), (5, 5), (5,), (5,))

In [17]:
#now get the fc layer weights
Wo, bo = model.fc.parameters()

In [18]:
Wo =Wo.data.numpy()
bo =bo.data.numpy()
Wo.shape, bo.shape

((2, 5), (2,))

In [19]:
# see if we can replicate the outputs
h_last = np.zeros(M)
x =X[0]
Yhats = np.zeros((T, K))

for t in range (T):
  h =np.tanh(x[t].dot(W_xh.T)+ b_xh + h_last.dot(W_hh.T) + b_hh)
  y= h.dot(Wo.T) + bo
  Yhats[t] = y

  h_last = h

print(Yhats)

[[ 0.48035506 -0.17040623]
 [ 1.02595769 -0.59066244]
 [ 0.58717667  0.14118948]
 [ 0.92349178 -0.31367282]
 [ 0.8062125  -0.32325623]
 [ 1.02682367 -0.46612537]
 [ 0.80365103 -0.38117828]
 [ 0.57484243 -0.09020649]
 [ 0.59660838 -0.25716497]
 [ 0.88540699 -0.26084363]]


In [20]:
#check
np.allclose(Yhats, Yhats_torch)

True